1. Bibliografía:
  - El **tema 6** permitirá al alumno familiarizarse con la programación genética (PG) 
  - y, **el mencionado artículo**, con una de las variantes de la PG, denominada Evolución Gramatical (del inglés Grammatical Evolution, GE). 
  - El **capítulo 8** describe distintos mecanismos para sintonizar de forma adaptativa cada uno de los diferentes parámetros de los que consta un algoritmo evolutivo (AE). 
  - El **capítulo 10** describe la forma de hibridar un AE con otros métodos de búsqueda. 
  - Finalmente, en el **capítulo 12**, se muestran distintas estrategias para manejar la existencia de restricciones en problemas de optimización que son abordados mediante AEs.
  
2. Secciones
  - Descripción del problema a resolver
  - Método para resolverlo 
    - se debe analizar la idoneidad o no del uso de GE para resolver el problema planteado
    - se debe incluir la expresión matemática de la función de evaluación finalmente empleada
    - se debe incluir la descripción de los diferentes operadores de inicialización, variación y selección empleados
    - se debe incluir la forma de manejar las restricciones, los mecanismos de control de parámetros utilizados, así como losmecanismos de búsqueda local implementados
  - Los resultados de los distintos experimentos realizados
  - Un análisis y comparación de resultados
  - Una sección de conclusiones
  - Una descripción del código implementado. 

3. Evaluación
  - Sobre la presentación (2/10)
    - Se evaluará especialmente la claridad en la redacción de la memoria y la capacidad de síntesis.  
  - Sobre el manejo de restricciones (1/10)
      - Se valorará la originalidad del mecanismo o mecanismos usados para el manejo de restricciones.
  - Sobre la configuración del algoritmo (2/10)
    - Aquí se valorará el procedimiento seguido por el alumno a la hora de elegir la mejor configuración de parámetros del algoritmo, incluyendo la implementación de mecanismos de control de parámetros adaptativos o auto-adaptativos
  - Sobre la hibridación del algoritmo con técnicas de búsqueda local (1/10)
    - Se valorará la originalidad del mecanismo de búsqueda local utilizado.
  - Sobre el análisis y comparación de resultados, y conclusiones (4/10)
    - Se valorará la forma de interpretar y comparar los diferentes experimentos realizados.
      - Es muy importante que dicha valoración se haga siempre en términos de los índices SR, MBF, AES 
      - y cualquier otra gráfica que considere oportuna como, por ejemplo, los plots de progreso de convergencia. 
    - Finalmente, se valorará la calidad de las conclusiones obtenidas a partir de la interpretación y comparación de resultados.


# Descripción del problema a resolver

Repitiendo las indicaciones dadas en el documento de la actividad, el problema consiste en implementar un algoritmo evolutivo para calcular la derivada simbólica de una función 
$$ f:X \subseteq \mathcal{R} \rightarrow \mathcal{R} $$ 

Disponemos de las siguientes dos definiciones:

> **Definición de derivada de una función en un punto**: Sea $X \subseteq \mathcal{R}$ un intervalo abierto. Diremos que $f:X \subseteq \mathcal{R} \rightarrow \mathcal{R}$ es derivable en $x_0 \in X$, denotado por $f'(x_0)$, si existe y es finito el límite:
$$
f'(x_0) = \lim \limits_{h \to 0} \frac{f(x_0+h)-f(x_0)}{h}  \tag{1}
$$

> **Definición de derivada de una función en un intervalo**: Sea $X \subseteq \mathcal{R}$ un intervalo abierto. Diremos que $f:X \subseteq \mathcal{R} \rightarrow \mathcal{R}$ es derivable en el intervalo $[a,b] \subseteq X$, si $f$ es derivable en cada uno de los puntos de dicho intervalo, es decir, si:
$$
f'(x) = \lim \limits_{h \to 0} \frac{f(x+h)-f(x)}{h}, \forall x \in [a,b]  \tag{2}
$$

Suponiendo que $f$ sea derivable en $[a,b]$, el problema de calcular la derivada lo vamos a transformar en un nuevo problema de optimización consistente en encontrar una función $g(x)$ que minimice la expresión:
$$
\min \limits_{g(x)} \frac{1}{b-a}\int_{a}^{b} error[f'(x),g(x)]dx \tag{3}
$$

dónde $f'(x)$ se calcularía utilizando la expresión $(2)$.

No obstante, el problema anterior se puede resolver de forma aproximada discretizando el intervalo de definición, es decir, cambiando el operador integral por un sumatorio:
$$
\min \limits_{g(x)} \frac{1}{N+1}\sum_{i=0}^{N} error_i[f'(a+i*h),g(a+i*h)] \tag{4}
$$
dónde $h=\frac{b-a}{N}$ es la anchura del subintervalo de muestreo para conseguir muestrear $N+1$ puntos en el intervalo $[a,b]$, y $f'(a+i*h)$ viene dado por:
$$
f'(a+i*h)=\frac{f(a+(i+1)*h) - f(a+i*h)}{h}, \forall i \in \{0,1,...,N\} \tag{5}
$$

# Método para resolverlo

